In [1]:
import numpy as np
from scipy.integrate import quad
from scipy import optimize, special
import pstats
from pstats import SortKey
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interactive

## Steady-state measures followed from Kolmogorov forward equations:

#### First moment = $\int_a^x \pi m(\pi)d \pi$;

#### Mass at maximum belief level = $m(1)$, and at the termination point = $n(\alpha)$;

#### Implied variable = $\Upsilon$;

#### Total mass = $\int_\alpha^x m(\pi) d \pi$;

#### And finally the surplus function. Note that we set $\psi(\pi) \equiv \pi^\psi$.

In [2]:
def firstMoment(p, a, kappa, phi, delta, lam, mu, psi):
    return(kappa * phi * p**psi / (mu * delta + kappa * phi * p**psi)
           * (delta / (delta + lam))
           * p * (1 - (a / (1 - a))**(delta / lam + 1) * (p / (1 - p))**(-(delta / lam + 1))))

def mOne(p, a, kappa, phi, delta, lam, mu, psi):
    return(kappa * phi * p**psi / (mu * delta + kappa * phi * p**psi)
          * (kappa * phi / (mu * (delta + lam) + kappa * phi))
          * (lam / (delta + lam))
          * p * (1 - (a / (1 - a))**(delta / lam + 1) * (p / (1 - p))**(-(delta / lam + 1))))

def nAlpha(p, a, kappa, phi, delta, lam, mu, psi):
    return(kappa * phi * p**psi / (delta * mu + kappa * phi * p**psi)
          * (a / p)**(delta / lam) * ((1 - a)/(1 - p))**(-(delta / lam + 1)))

def upsilon(i, j, x, y, delta, lam):
    return((y / x)**(delta / lam - 1) * ((1 - y)/(1 - x))**(-(delta / lam + 2)) * y**i * (1-y)**j 
           - x**i * (1 - x)**j)

def totalMass(p, a, kappa, phi, delta, lam, mu, psi): 
    return(kappa * phi * p**psi / (mu * delta + kappa * phi * p**psi)
          * (p - a) / (upsilon(2, 1, a, p, delta, lam) - a * upsilon(1, 1, a, p, delta, lam))
          * (upsilon(1, 1, a, p, delta, lam) - (lam / (lam + delta) * upsilon(2, 1, a, p, delta, lam))))

def surplus(a, p, kappa, phi, delta, lam, c, mu, psi):
    return((lam - c) * mOne(p, a, kappa, phi, delta, lam, mu, psi) 
           + lam * firstMoment(p, a, kappa, phi, delta, lam, mu, psi) 
           - c * totalMass(p, a, kappa, phi, delta, lam, mu, psi))

#### Defining the integral $\int_\alpha^p \psi(\pi)m(\pi)d \pi$, and the steady-state average reputation score $M\equiv E[\psi(\pi_\infty)]$:

In [3]:
def psiMoment(p, a, kappa, phi, delta, lam, mu, psi):
    return ((delta * kappa * phi / ((delta + lam) * (delta * mu + p**psi * kappa * phi))) 
            * (p**(psi - delta / lam) 
            * special.hyp2f1(-(1 + delta / lam), 1 - delta / lam - psi, -delta / lam, 1 - p)
            - p**psi * a**(-delta / lam) * ((a / p)**(delta / lam)) * ((1 - a) / (1 - p))**(-(1 + delta / lam))
            * special.hyp2f1(-(1 + delta / lam), 1 - delta / lam - psi, -delta / lam, 1 - a)))
    

def avgReputation(p, a, kappa, phi, delta, lam, mu, psi):
    return((1 + mu * (delta + lam) / (kappa * phi)) * mOne(p, a, kappa, phi, delta, lam, mu, psi)
           + p**psi * delta * mu / (delta * mu + kappa * phi * p**psi)
           + psiMoment(p, a, kappa, phi, delta, lam, mu, psi)
           + a**psi * nAlpha(p, a, kappa, phi, delta, lam, mu, psi))


#### Finding the fixed-point of the equation $\mu=M(\mu,\ldots)$.

In [4]:
def M_fixed_point(mu, p, a, kappa, phi, delta, lam, psi):
    return(avgReputation(p, a, kappa, phi, delta, lam, mu, psi)-mu)

def mu_fixed_point(p, a, kappa, phi, delta, lam, psi):
    return(optimize.root_scalar(M_fixed_point,args=(p, a, kappa, phi, delta, lam, psi),x0=p/2,x1=p/3).root)

#### The equilibrium $\alpha_e$ and the the socially optimal  $\alpha_*$ are as followed:

In [5]:
def Alpha(kappa, r, phi, delta, lam, c, mu):
    w = (r + delta)**(-1) * kappa * phi * (lam - c) / (mu * (r + delta + lam) + kappa * phi)
    return(c / (lam * (1 + w)))

def M_equil_fixed_point(mu, p, kappa, r, phi, delta, lam, c, psi):
    return(avgReputation(p, Alpha(kappa, r, phi, delta, lam, c, mu), kappa, phi, delta, lam, mu, psi) - mu)

def mu_equil(p, kappa, r, phi, delta, lam, c, psi):
    return(optimize.root_scalar(M_equil_fixed_point,args=(p, kappa, r, phi, delta, lam, c, psi),x0=p/2,x1=p/3).root)

def my_func(a, p, kappa, phi, delta, lam, c, psi):
    return(-surplus(a, p, kappa, phi, delta, lam, c,  mu_fixed_point(p, a, kappa, phi, delta, lam, psi), psi))

def Alpha_star(p, kappa, phi, delta, lam, c, psi):
    return optimize.minimize(my_func,p/2, args=(p, kappa, phi, delta, lam, c, psi), bounds=[(0.01,p)]).x[0]

## Interactive plots:

### Social surplus as a function of the reputation cutoff $\alpha$:

In [6]:
## (Un)comment the following line to set the varaibles
p = 0.4; phi = 1.4; delta = 0.3; lam = 1.75; c = 0.7; r = 0.8; kappa = 1

In [7]:
def surplus_manipulate(psi,mu):
    plt.figure(2)
    a = np.linspace(0.001, p, 50, endpoint=False)
    plt.plot(a, surplus(a, p, kappa, phi, delta, lam, c, mu, psi))
    plt.ylim(0, 0.2)
    plt.title('Social Surplus as a function of the reputation cutoff')
    plt.ylabel('Steady-state social surplus')
    plt.show()
    
interactive_plot = interactive(surplus_manipulate, psi=(0.1,0.9,0.05),mu=(0.1,0.4,0.02))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='psi', max=0.9, min=0.1, step=0.05), FloatSlider(valu…

### Steady-state reputation score $\mu$ as a function of the reputation cutoff $\alpha$:

In [8]:
def mu_fixed_point_manipulate(p, phi, delta, lam, psi):
    plt.figure(2)
    a = np.linspace(0.001, p, 50, endpoint=False)
    y = np.asarray([mu_fixed_point(p, x, 1, phi, delta, lam, psi) for x in a])
    plt.plot(a, y)
    plt.ylim(0, 0.9)
    plt.title('Steady-state $\mu$ as a function of the reputation cutoff')
    plt.xlabel('Cutoff')
    plt.ylabel('Steady-state $\mu$')
    plt.show()
    
interactive_plot = interactive(mu_fixed_point_manipulate, p=(0.3,0.7,0.05), phi=(0.2,1.2,0.1), delta=(0.01,0.5,0.02),
                               lam=(1,3,0.2), psi=(0.1,0.9,0.05))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='p', max=0.7, min=0.3, step=0.05), FloatSlider(value=…

### Equilibrium vs. socially optimal *reputation cutoff* as a function of the curvature of the matching function $\psi$:

In [9]:
def Alpha_comp(p, r, phi, delta, lam, c):
    plt.figure(2)
    psi = np.linspace(0.1,0.9,100)
    Alpha_e = np.asarray([Alpha(1, r, phi, delta, lam, c, mu_equil(p, 1, r, phi, delta, lam, c, x)) for x in psi])
    Alpha_s = np.asarray([Alpha_star(p, 1, phi, delta, lam, c, x) for x in psi ])
    plt.plot(psi, Alpha_e, 'b', label='equilibrium $\\alpha$')
    plt.plot(psi, Alpha_s, 'r', label='socially optimal $\\alpha$')
    plt.legend()
    plt.ylim(0, 0.2)
    plt.show()  
    
    
interactive_plot = interactive(Alpha_comp, p=(0.3,0.7,0.05), r=(0.5,0.9,0.05), phi=(0.2,1.2,0.1),
                               delta=(0.05,0.4,0.02),lam=(1,3,0.2), c=(0.3,0.5,0.05))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='p', max=0.7, min=0.3, step=0.05), FloatSlider(value=…

### Equilibrium vs. socially optimal *surplus* as a function of the curvature of the matching function $\psi$:

In [10]:
def surplus_comp(p, r, phi, delta, lam, c):
    plt.figure(2)
    psi = np.linspace(0.1,0.9,100)
    s_e = np.asarray([surplus(Alpha(1, r, phi, delta, lam, c, mu_equil(p, 1, r, phi, delta, lam, c, x)), p, 1,
                              phi, delta, lam, c, mu_equil(p, 1, r, phi, delta, lam, c, x), x) for x in psi])
    s_s = np.asarray([surplus(Alpha_star(p, 1, phi, delta, lam, c, x), p, 1, phi, delta, lam, c,
                              mu_fixed_point(p, Alpha_star(p, 1, phi, delta, lam, c, x), 1, phi, delta, lam, x),
                              x) for x in psi])
 
    plt.plot(psi,s_e,'b', label='equilibrium surplus')
    plt.plot(psi,s_s,'r', label='optimal surplus')
    plt.legend()
    plt.ylim(0, 0.2)
    plt.show()  
    
interactive_plot = interactive(surplus_comp, p=(0.3,0.7,0.05), r=(0.5,0.9,0.05), phi=(0.2,1.2,0.1),
                               delta=(0.05,0.4,0.02),lam=(1,3,0.2), c=(0.3,0.7,0.05))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='p', max=0.7, min=0.3, step=0.05), FloatSlider(value=…